In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

  ### DECISION TREE

In [2]:
df_casual = pd.read_pickle('../datos/bikes_casual_estand2.pkl')
df_casual.head()

,temperatura,velocidad_viento,estacion,año,mes,festivo,dia_semana,dia_laboral,tiempo,cliente_casual
1,-0.425690,0.687550,1,1,1.0,1,2.0,2,3,331
2,-0.949970,0.685210,1,1,1.5,3,1.0,1,3,131
3,-0.938563,-0.210435,1,1,3.0,3,1.0,1,4,120
4,-0.853992,0.060294,2,1,3.0,3,2.0,2,4,108
5,-0.924922,-0.930212,2,1,5.0,3,1.0,1,4,82


In [3]:
# Dividimos los datos y ajustamos el modelo

X = df_casual.drop("cliente_casual", axis = 1)
y = df_casual["cliente_casual"]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

arbol = DecisionTreeRegressor(random_state =0)

arbol.fit(x_train, y_train)

y_pred_test_dt = arbol.predict(x_test)
y_pred_train_dt = arbol.predict(x_train)

In [4]:
# Sacamos el nº de variables predictoras(3) y la profundidad(22)

max_features = np.sqrt(len(x_train.columns))
print(max_features)

print(arbol.tree_.max_depth)

3.0
24


In [5]:
param_rf = {"max_depth": [6, 7, 8, 9, 10],  
        "max_features": [2, 3, 4, 5, 6],      
        "min_samples_split": [10, 20, 30, 50],
        "min_samples_leaf": [10, 15, 20, 30]}

In [6]:
param = {"max_depth": [4, 8, 6, 10, 12],  
        "max_features": [1,2,3],      
        "min_samples_split": [10, 30, 50, 100],
        "min_samples_leaf": [10, 30, 50, 100]} 

In [7]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param_rf, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")

In [ ]:
gs.fit(x_train, y_train)

In [ ]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=10, max_features=6, min_samples_leaf=15,
                      min_samples_split=10)

In [ ]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [6]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [ ]:
dt_results = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "Decision tree")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,315.809589,215244.709759,463.944727,0.559571,test,Decision tree
1,303.488477,188206.729484,433.827995,0.595844,train,Decision tree


> Las métricas sacadas en el modelo de Decision Tree son muy inferiores a 0,7, por lo que pasamos a realizar el modelo Random Forest para poder mejorarlas.

  ### RANDOM FOREST

In [7]:
forest = RandomForestRegressor(random_state =42)

forest.fit(x_train, y_train)

y_pred_test_rf = forest.predict(x_test)
y_pred_train_rf= forest.predict(x_train)

In [8]:
rf_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest")
rf_results

,MAE,MSE,RMSE,R2,set,modelo
0,310.641986,207963.976276,456.030675,0.574469,test,Random Forest
1,129.355386,33350.290215,182.620618,0.928384,train,Random Forest


> Con el modelo Random Forestr tenemos mejores datos, no obstante, siguen siendo bastante malos por lo que procedemos a revisar los hiperparámetros.

- he probado con estos hiperparámetros:
param2 = {"max_depth": [4, 8, 6, 10],  
        "max_features": [1,2,3],      
        "min_samples_split": [10, 15, 20, 30],
        "min_samples_leaf": [10, 15, 20, 30]} 
los voy a modificar para probar otra vez
las métricas era
R2 TEST > 0.43
R2 TRAIN > 0.57

In [ ]:
break

In [19]:
param = {"max_depth": [6, 7, 8, 9, 10],  
        "max_features": [2, 3, 4, 5, 6],      
        "min_samples_split": [10, 20, 30, 50],
        "min_samples_leaf": [10, 15, 20, 30],
        "random_state":  [42]}

In [20]:
gs = GridSearchCV(
            estimator=RandomForestRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")

- con los hiperparámetros de Cassia

In [21]:
gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [6, 7, 8, 9, 10],
                         'max_features': [2, 3, 4, 5, 6],
                         'min_samples_leaf': [10, 15, 20, 30],
                         'min_samples_split': [10, 20, 30, 50],
                         'random_state': [42]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [22]:
mejor_modelo_rf = gs.best_estimator_
mejor_modelo_rf

RandomForestRegressor(max_depth=6, max_features=6, min_samples_leaf=10,
                      min_samples_split=10, random_state=42)

In [23]:
y_pred_test_rf2 = mejor_modelo_rf.predict(x_test)
y_pred_train_rf2= mejor_modelo_rf.predict(x_train)

In [24]:
rf_results2 = metricas(y_test, y_train, y_pred_test_rf2, y_pred_train_rf2, "Random Forest 2")
rf_results2

,MAE,MSE,RMSE,R2,set,modelo
0,298.856024,187091.382892,432.540614,0.617178,test,Random Forest 2
1,297.939492,174350.125907,417.552543,0.625600,train,Random Forest 2


> ### Métricas malísimas de nuevo !! REvisar!!


SACAMOS CON RANDOM FOREST NUEVAS MÉTRICAS PERO ELIMINANDO UNA VARIABLE PREDICTORA

In [9]:
df_casual.head(2)

,temperatura,velocidad_viento,estacion,año,mes,festivo,dia_semana,dia_laboral,tiempo,cliente_casual
1,-0.42569,0.68755,1,1,1.0,1,2.0,2,3,331
2,-0.94997,0.68521,1,1,1.5,3,1.0,1,3,131


In [10]:
X1 = df_casual.drop(["cliente_casual", 'velocidad_viento', 'estacion'], axis = 1)
y1 = df_casual["cliente_casual"]

DECISION TREE

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.2, random_state = 42)

arbol = DecisionTreeRegressor(random_state =0)

arbol.fit(x_train, y_train)

y_pred_test_dt = arbol.predict(x_test)
y_pred_train_dt = arbol.predict(x_train)

In [12]:
dt_results01 = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "Decision tree")
dt_results01

,MAE,MSE,RMSE,R2,set,modelo
0,440.239726,414148.565068,643.543755,0.152579,test,Decision tree
1,3.818182,1083.283019,32.913265,0.997674,train,Decision tree


In [37]:
param_dt = {"max_depth": [6, 7, 8, 9, 10],  
        "max_features": [2, 3, 4, 5, 6],      
        "min_samples_split": [10, 20, 30, 50],
        "min_samples_leaf": [10, 15, 20, 30]}

In [38]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param_dt, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")

In [39]:
gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [6, 7, 8, 9, 10],
                         'max_features': [2, 3, 4, 5, 6],
                         'min_samples_leaf': [10, 15, 20, 30],
                         'min_samples_split': [10, 20, 30, 50]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [40]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=7, max_features=5, min_samples_leaf=10,
                      min_samples_split=20)

In [41]:
y_pred_test_dt0 = mejor_modelo.predict(x_test)
y_pred_train_dt0 = mejor_modelo.predict(x_train)

In [45]:
dt_results0 = metricas(y_test, y_train, y_pred_test_dt0, y_pred_train_dt0, "Decision tree1")
dt_results0

,MAE,MSE,RMSE,R2,set,modelo
0,296.138970,181935.939416,426.539493,0.627727,test,Decision tree1
1,290.635401,176425.460748,420.030309,0.621144,train,Decision tree1


In [54]:
max_features = np.sqrt(len(x_train.columns))
print(max_features)

print(arbol.tree_.max_depth)

2.6457513110645907
23


RANDOM FOREST

In [13]:
forest = RandomForestRegressor(random_state =65)

forest.fit(x_train, y_train)

y_pred_test_rf = forest.predict(x_test)
y_pred_train_rf= forest.predict(x_train)

In [14]:
rf_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest")
rf_results

,MAE,MSE,RMSE,R2,set,modelo
0,306.966716,179494.606166,423.668038,0.632722,test,Random Forest
1,136.494347,38099.925991,195.192023,0.918184,train,Random Forest


- HE PROBADO A QUITAR UN PAR DE VARIABLES PREDICTORAS Y A PROBAR CON LOS HIPERPARÁMETROS DEL DECISION TREE PERO LAS MÉTRICAS EN EL RANDOM 2 SON PEORES: 0.62 - 0.59
- QUIZÁ MODIFICANDO LOS HIPERPARÁMETROS DE NUEVO PODAMOS MEJORARLAS O QUITANDO ALGUNA VARIABLE PREDICTORA MÁS PARA QUE EL PRIMER RANDOM MEJORE UN POCO. NO SÉ.

In [61]:
param = {"max_depth": [6, 7, 8, 9, 10],  
        "max_features": [1, 2, 3],      
        "min_samples_split": [10, 20, 30, 50],
        "min_samples_leaf": [10, 15, 20, 30],
        "random_state":  [65]}

In [62]:
gs1 = GridSearchCV(
            estimator=RandomForestRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")

In [63]:
gs1.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [6, 7, 8, 9, 10],
                         'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 15, 20, 30],
                         'min_samples_split': [10, 20, 30, 50],
                         'random_state': [65]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [64]:
mejor_modelo_rf1 = gs1.best_estimator_
mejor_modelo_rf1

RandomForestRegressor(max_depth=7, max_features=3, min_samples_leaf=10,
                      min_samples_split=10, random_state=65)

In [65]:
y_pred_test_rf2 = mejor_modelo_rf1.predict(x_test)
y_pred_train_rf2= mejor_modelo_rf1.predict(x_train)

In [66]:
rf_results3 = metricas(y_test, y_train, y_pred_test_rf2, y_pred_train_rf2, "Random Forest 1")
rf_results3

,MAE,MSE,RMSE,R2,set,modelo
0,298.966583,182230.650210,426.884821,0.627124,test,Random Forest 1
1,310.181382,187580.180706,433.105277,0.597190,train,Random Forest 1


PROBAMOS ELIMINANDO OTRA VARIABLE PREDICTORA

In [15]:
X2= df_casual.drop(["cliente_casual", 'velocidad_viento', 'estacion', 'dia_laboral'], axis = 1)
y2 = df_casual["cliente_casual"]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X2, y2, test_size = 0.2, random_state = 42)

arbol = DecisionTreeRegressor(random_state =0)

arbol.fit(x_train, y_train)

y_pred_test_dt = arbol.predict(x_test)
y_pred_train_dt = arbol.predict(x_train)

In [17]:
dt_results4 = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "Decision tree4")
dt_results4

,MAE,MSE,RMSE,R2,set,modelo
0,431.821918,405078.599315,636.457854,0.171137,test,Decision tree4
1,7.178388,2621.798456,51.203500,0.994370,train,Decision tree4


In [20]:
forest = RandomForestRegressor(random_state =65)

forest.fit(x_train, y_train)

y_pred_test_rf4 = forest.predict(x_test)
y_pred_train_rf4= forest.predict(x_train)

In [21]:
rf_results4r = metricas(y_test, y_train, y_pred_test_rf4, y_pred_train_rf4, "Random Forest4")
rf_results4r

,MAE,MSE,RMSE,R2,set,modelo
0,317.303614,193976.403053,440.427523,0.603090,test,Random Forest4
1,140.158724,40977.867395,202.429907,0.912004,train,Random Forest4
